In [ ]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
# IMAGE_DIR = os.path.join(ROOT_DIR, "images")
IMAGE_DIR = os.path.join(ROOT_DIR, "../cvpr/input/test")

In [ ]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


In [ ]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

In [ ]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

# need: car, motorcycle, bicycle, pedestrian, truck, bus, and tricycle


# 3: car -> 33
# 4: motorcycle -> 34
# 2: bicycle -> 35
# 1: person -> 36
# 8: truck -> 38
# 6: bus -> 39
# tricycle -> none

In [ ]:
def get_pcount_encode(image, x_min, y_min, shape):
    pcount = 0
    encoded = ""
    
    for y in range(len(image)):
        start, end, flag = 0, 0, True
        for e in range(len(image[y])):
            if image[y][e]:
                if flag:
                    start, flag = e, False
                end = e
        if start != 0:
            encoded += "{} {}|".format(((x_min + y) * shape[1]) + y_min + start, end - start + 1)
            pcount += end - start + 1
    return pcount, encoded

In [ ]:
file_names = next(os.walk(IMAGE_DIR))[2]
data = []
for idx in range(len(file_names)):
    img_i = file_names[idx]
    img = skimage.io.imread(os.path.join(IMAGE_DIR, img_i))
    res = model.detect([img], verbose=1)
    print(f'detect {idx} over!')
    r = res[0]
    shape = r['masks'].shape
    for i in range(len(r['class_ids'])):
        label_id = id_transfer(r['class_ids'][i])
        if label_id is not None:
            obj = dict()
            obj['ImageId'] = img_i.split('.')[0]
            obj['LabelId'] = label_id
            obj['Confidence'] = r['scores'][i]
            
            
            x_min, y_min, x_max, y_max = r['rois'][i]
            tmp = r['masks'][x_min:x_max, y_min:y_max, i]
            pcount, encode = get_pcount_encode(tmp, x_min, y_min, shape)
            
            ''' slower version
            pcount, encode = get_pcount_encode(r['masks'], i)
            '''            
            obj['PixelCount'] = pcount
            obj['EncodedPixels'] = encode
            data.append(obj)
    print(f"finish {idx}/1916")